In [1]:
import os
from PIL import Image
import numpy as np
np.random.seed(2016)
from keras.layers.convolutional import (Convolution3D, MaxPooling3D,ZeroPadding3D,Convolution2D,MaxPooling2D)
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import LSTM, GRU, SimpleRNN
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.regularizers import l2, activity_l2

Using TensorFlow backend.


In [2]:
def load_data():
    data = np.empty((2587,9,3,64,48),dtype="float32")
    label = np.empty((2587,),dtype="uint8")
    imgPath='/Users/baidu/Documents/workspace/Action_recognition_scripts_UCF101/data/tmp_frames/train/'
    f = open('/Users/baidu/Documents/workspace/Action_recognition_scripts_UCF101/data/ucfTrainTestlist/trainlist05.txt')
    vids = f.readlines()
    vids = [video.rstrip() for video in vids]
    
    for i in range(len(vids)):
        keypath=vids[i].split('/')[1].split('.')[0]
        value=vids[i].split(' ')[1]
        label[i]=value
        #print keypath
        #print value
        for j in range(1,10):
            path=imgPath+keypath+'_0'+str(j)+'.jpeg'
            if os.path.exists(path):
                img=Image.open(imgPath+keypath+'_0'+str(j)+'.jpeg')
                img=img.resize((64,48))
                #print imgPath+keypath+'_0'+str(j)+'.jpeg'
                arr = np.asarray(img, dtype='float32')
                data[i,j-1,:,:,:]=(arr.T-128)/128
            else:
                data[i,j-1,:,:,:]=data[i,j-2,:,:,:]
            
    return np.transpose(np.nan_to_num(data),(0,2,1,3,4)),label

In [3]:
data,midlabel=load_data()
nb_classes=20
label = np_utils.to_categorical(midlabel-1, nb_classes)

data1=data
label1=label

data2=np.transpose(data,(0,2,1,3,4))
label2=label

In [4]:
def load_test_data():
    data = np.empty((100,9,3,64,48),dtype="float32")
    label = np.empty((100,),dtype="uint8")
    imgPath='/Users/baidu/Documents/workspace/Action_recognition_scripts_UCF101/data/tmp_frames/test/'
    f = open('/Users/baidu/Documents/workspace/Action_recognition_scripts_UCF101/data/ucfTrainTestlist/testlist01_100.txt')
    vids = f.readlines()
    vids = [video.rstrip() for video in vids]
    
    for i in range(len(vids)):
        keypath=vids[i].split('/')[1].split('.')[0]
        value=vids[i].split(' ')[1]
        label[i]=value
        #print keypath
        #print value
        for j in range(1,10):
            path=imgPath+keypath+'_0'+str(j)+'.jpeg'
            if os.path.exists(path):
                img=Image.open(imgPath+keypath+'_0'+str(j)+'.jpeg')
                img=img.resize((64,48))
                #print imgPath+keypath+'_0'+str(j)+'.jpeg'
                arr = np.asarray(img, dtype='float32')
                data[i,j-1,:,:,:]=(arr.T-128)/128
            else:
                data[i,j-1,:,:,:]=data[i,j-2,:,:,:]
            
    return np.transpose(np.nan_to_num(data),(0,2,1,3,4)),label

data_test,midlabel_test=load_test_data()
label_test = np_utils.to_categorical(midlabel_test-1, nb_classes)
data2_test=np.transpose(data_test,(0,2,1,3,4))
label2_test=label_test
print data2_test.shape
print label2_test.shape

(100, 9, 3, 64, 48)
(100, 20)


In [ ]:
#一种策略是使用TimeDistributed包裹2D卷积 参考：https://github.com/fchollet/keras/issues/2646
model2 = Sequential()
model2.add(TimeDistributed(Convolution2D(128, 3, 3,border_mode='valid'),input_shape=(9, 3, 64, 48)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model2.add(Dropout(0.4))

model2.add(TimeDistributed(Convolution2D(64, 3, 3, 
                                         border_mode='valid',
                                         W_regularizer=l2(0.01),
                                         activity_regularizer=activity_l2(0.01))))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model2.add(Dropout(0.4))

model2.add(TimeDistributed(Convolution2D(128, 3, 3, 
                                         border_mode='valid',
                                         W_regularizer=l2(0.01),
                                         activity_regularizer=activity_l2(0.01))))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model2.add(Dropout(0.4))

model2.add(TimeDistributed(Flatten()))
model2.add(TimeDistributed(Dense(256,activation='relu',
                                 W_regularizer=l2(0.01),activity_regularizer=activity_l2(0.01))))
model2.add(TimeDistributed(Dense(128,activation='relu',
                                 W_regularizer=l2(0.01),activity_regularizer=activity_l2(0.01))))

model2.add(LSTM(output_dim=256,return_sequences=True))
model2.add(LSTM(output_dim=128))
model2.add(Dense(20,activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])
model2.fit(data2, label2,
              batch_size=64,
              nb_epoch=20,shuffle=True,validation_data=(data2_test, label2_test))

Train on 2587 samples, validate on 100 samples
Epoch 1/20
 832/2587 [========>.....................] - ETA: 1491s - loss: 2.9933 - acc: 0.0781

In [41]:
#一种策略是完全使用3D卷积
model = Sequential()
# 1st layer group
model.add(Convolution3D(16, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv1',
                        input_shape=(3, 9, 64, 48)))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                        border_mode='valid', name='pool1'))
#model.add(LSTM(64))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))
model.add(Flatten())
model.add(Dense(100, init='normal', activation='relu'))
model.add(Dense(2, init='normal'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(data1, label1,
              batch_size=2,
              nb_epoch=2,
              shuffle=True,
         ,validation_data=(data2_test, label2_test))

In [6]:
score = model2.evaluate(data2_test, label2_test, verbose=1)
print score
model2.metrics_names

100/100 [==============================] - 4s     
[1.9148659229278564, 0.38]


['loss', 'acc']

In [10]:
c3d_model = Sequential()
    # 1st layer group
c3d_model.add(Convolution3D(64, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv1',
                            subsample=(1, 1, 1),
                            input_shape=(3, 16, 112, 112)))
c3d_model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                           border_mode='valid', name='pool1'))

    # 2nd layer group
c3d_model.add(Convolution3D(128, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv2',
                            subsample=(1, 1, 1)))
c3d_model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool2'))

    # 3rd layer group
c3d_model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv3a',
                            subsample=(1, 1, 1)))
c3d_model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv3b',
                            subsample=(1, 1, 1)))
c3d_model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool3'))

    # 4th layer group
c3d_model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv4a',
                            subsample=(1, 1, 1)))
c3d_model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv4b',
                            subsample=(1, 1, 1)))
c3d_model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool4'))

    # 5th layer group
c3d_model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv5a',
                            subsample=(1, 1, 1)))
c3d_model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                            border_mode='same', name='conv5b',
                            subsample=(1, 1, 1)))
c3d_model.add(ZeroPadding3D(padding=(0, 1, 1)))
c3d_model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           border_mode='valid', name='pool5'))
c3d_model.add(Flatten())

    # FC layers group
c3d_model.add(Dense(4096, activation='relu', name='fc6'))
c3d_model.add(Dropout(.5))
c3d_model.add(Dense(4096, activation='relu', name='fc7'))
c3d_model.add(Dropout(.5))
c3d_model.add(Dense(487, activation='softmax', name='fc8'))


# @fchollet提供的一种解决方案
参考：https://github.com/fchollet/keras/issues/129
Put all the pictures in a sequence in a single batch
insert the following custom layer after your convolution + flatten stage:
class MyReshape(Layer):
    def get_output(self, train):
        X = self.get_input(train)
        nshape = (1,) + X.shape 
        return theano.tensor.reshape(X, nshape)
It turns a batch of N vectors into a batch of size 1 containing a sequence of N vectors.

Note that if you do something like that, the length of the input and of the labels won't match, so you won't be able to use the model.fit() method for training. However, you should be able to use the model.train(X, y) method just fine, on small minibatches.